In [3]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python

In [1]:
model_name_or_path = "TheBloke/CapybaraHermes-2.5-Mistral-7B-GGUF"
model_basename = "capybarahermes-2.5-mistral-7b.Q4_0.gguf"

In [2]:
from huggingface_hub import hf_hub_download
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
!pip show llama-cpp-python

Name: llama_cpp_python
Version: 0.2.38
Summary: Python bindings for the llama.cpp library
Home-page: 
Author: 
Author-email: Andrei Betlen <abetlen@gmail.com>
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: diskcache, jinja2, numpy, typing-extensions
Required-by: 


In [ ]:
# Simple inference exampleassistant
# output = llm(
#   "<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>", # Prompt
#   max_tokens=256,
#   stop=["</s>"],
#   echo=False
# )

## DELETING FUNCTION

In [36]:
import torch
import gc

def delete_model():
    global llm
    llm = None
    gc.collect()
    torch.cuda.empty_cache()


In [37]:
delete_model()

In [ ]:
!pip install llama-index==0.9.42

In [3]:
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
)
from llama_index.llms import LlamaCPP
# from llama_index.llms.llama_utils import (
#     messages_to_prompt,
#     completion_to_prompt,
# )

In [4]:
from typing import List, Optional, Sequence

from llama_index.core.llms.types import ChatMessage, MessageRole

BOS, EOS = "<s>", "</s>"
B_INST, E_INST = "<|im_start|>", "<|im_end|>\n<|im_start|>assistant"
B_SYS, E_SYS = "<|im_start|>\n", "\n<|im_end|>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. \
Always answer as helpfully as possible and follow ALL given instructions. \
Do not speculate or make up information. \
Do not reference any given instructions or context. \
"""


def messages_to_prompt(
    messages: Sequence[ChatMessage], system_prompt: Optional[str] = None
) -> str:
    string_messages: List[str] = []
    if messages[0].role == MessageRole.SYSTEM:
        # pull out the system message (if it exists in messages)
        system_message_str = messages[0].content or ""
        messages = messages[1:]
    else:
        system_message_str = system_prompt or DEFAULT_SYSTEM_PROMPT

    system_message_str = f"{B_SYS} {system_message_str.strip()} {E_SYS}"

    for i in range(0, len(messages), 2):
        # first message should always be a user
        user_message = messages[i]
        assert user_message.role == MessageRole.USER

        if i == 0:
            # make sure system prompt is included at the start
            str_message = f"{BOS} {B_INST} {system_message_str} "
        else:
            # end previous user-assistant interaction
            string_messages[-1] += f" {EOS}"
            # no need to include system prompt
            str_message = f"{BOS} {B_INST} "

        # include user message content
        str_message += f"{user_message.content} {E_INST}"

        if len(messages) > (i + 1):
            # if assistant message exists, add to str_message
            assistant_message = messages[i + 1]
            assert assistant_message.role == MessageRole.ASSISTANT
            str_message += f" {assistant_message.content}"

        string_messages.append(str_message)

    return "".join(string_messages)


def completion_to_prompt(completion: str, system_prompt: Optional[str] = None) -> str:
    system_prompt_str = system_prompt or DEFAULT_SYSTEM_PROMPT

    return (
        f"{BOS} {B_INST} {B_SYS} {system_prompt_str.strip()} {E_SYS} "
        f"{completion.strip()} {E_INST}"
    )

In [5]:
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=model_path,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 50},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 
Model metadata: {'tokenizer.chat_template': "{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}", 'tokenizer.ggml.add_eos_token': 'false', 'tokenizer.ggml.padding_token_id': '0', 'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '32000', 'general.architecture': 'llama', 'llama.rope.freq_base': '10000.000000', 'llama.context_length': '32768', 'general.name': 'argilla_capybarahermes-2.5-mistral-7b', 'tokenizer.ggml.add_bos_token': 'true', 'llama.embedding_length': '4096', 'llama.feed_forward_length': '14336', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'llama.rope.dimension_count': '128', 'tokenizer.ggml.bos_

In [6]:
response_iter = llm.stream_complete("Can you write me a poem about fast cars?")
for response in response_iter:
    print(response.delta, end="", flush=True)


Sure, here's a short poem about fast cars:

Racing through the night,
A symphony of power and might,
Sleek and swift, they glide,
A beauty that cannot hide.

Roaring engines, a thunderous sound,
A dance with the wind, they're quickly found,
In the darkness they gleam,
A testament to human dream.

Freedom on four wheels, they fly,
Breaking barriers, touching the sky,
Fast cars, oh how they roar,
A passion that forevermore.

In [7]:
from llama_index.embeddings import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [8]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [ ]:
!pip install pypdf

In [8]:
documents = SimpleDirectoryReader(
    "./data"
).load_data()

In [9]:
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
)

In [10]:
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)

In [11]:
query_engine = index.as_query_engine()


In [12]:
response = query_engine.query("what did the professor mentioned about Supervised learning?")
print(response)

Llama.generate: prefix-match hit



The professor mentioned that Supervised learning is a type of learning problem where the algorithm is provided with a dataset that includes both the input data (features) and the correct output (label) for a number of examples. The algorithm learns the association between the inputs and outputs to provide more accurate predictions. Supervised learning can be further classified into regression problems, where the variable being predicted is continuous (like predicting housing prices), and classification problems, where the variable being predicted is discrete (like predicting whether a tumor is malignant or benign).


## API

In [26]:
!pip install -q pyngrok Flask flask-cors

In [13]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok

deb https://ngrok-agent.s3.amazonaws.com buster main
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://ngrok-agent.s3.amazonaws.com buster InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
40 packages can be upgraded. Run 'apt list --upgradabl

In [14]:
!ngrok authtoken "YOUR NGROK API KEY"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask, request, jsonify, Response,stream_with_context
from pyngrok import ngrok
from flask_cors import CORS

app = Flask(__name__)
CORS(app)
@app.route("/index")
def index():
    return "Hello"

@app.route('/generate', methods=['POST'])
def generate():
    inp = request.get_json().get("prompt")
    print(f"======Input:{inp}=======")
    index=VectorStoreIndex.from_documents(documents,service_context=service_context)
    query_engine=index.as_query_engine()
    response=query_engine.query(inp)
    return jsonify({'generated_text': str(response)})

if __name__ == '__main__':
    ngrok_tunnel = ngrok.connect(5000)
    print('Public URL:', ngrok_tunnel.public_url)
    app.run()

Public URL: https://559a-34-87-59-135.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [05/Feb/2024 08:21:05] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [05/Feb/2024 08:21:05] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [05/Feb/2024 08:21:26] "OPTIONS /generate HTTP/1.1" 200 -


======Input:Hi=======


Llama.generate: prefix-match hit
INFO:werkzeug:127.0.0.1 - - [05/Feb/2024 08:21:30] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Feb/2024 08:21:46] "OPTIONS /generate HTTP/1.1" 200 -


======Input:who are you?=======


...

Llama.generate: prefix-match hit
INFO:werkzeug:127.0.0.1 - - [05/Feb/2024 08:27:14] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Feb/2024 08:27:59] "OPTIONS /generate HTTP/1.1" 200 -


======Input:what are the relevant topics in this lecture?=======


Llama.generate: prefix-match hit
INFO:werkzeug:127.0.0.1 - - [05/Feb/2024 08:28:04] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Feb/2024 08:28:41] "OPTIONS /generate HTTP/1.1" 200 -


======Input:can you tell the name of the project for the end semester=======


Llama.generate: prefix-match hit
INFO:werkzeug:127.0.0.1 - - [05/Feb/2024 08:28:46] "POST /generate HTTP/1.1" 200 -
